In [1]:
import polars as pl

import rustystats as rs

data = pl.read_parquet("https://raw.githubusercontent.com/PricingFrontier/pricing-data-example/917c853e256df8d5814721ab56f72889a908bb08/data/processed/frequency_set.parquet")

train_data = data.filter(pl.col("Group") <= "3")
test_data = data.filter(pl.col("Group") == "4")
holdout_data = data.filter(pl.col("Group") == "5")

In [2]:
terms = {
        'BonusMalus': {'type': 'ns'}, 
        'VehAge': {'type': 'ns'},
        'DrivAge': {'type': 'ns'}, 
        'VehPower': {'type': 'ns'}, 
        'Area': {'type': 'target_encoding'},
        'Region': {'type': 'target_encoding'},
        'VehGas': {'type': 'target_encoding'},
        'VehBrand': {'type': 'target_encoding'}
    }

interactions=[{
        "VehPower": {"type": "linear"},
        "Region": {"type": "categorical"},
        "target_encoding": True,
        "include_main": False
    }]

result = rs.glm_dict(
    response='ClaimCount',
    terms=terms,
    interactions=interactions,
    data=data,
    family='poisson',
    offset ='Exposure'
).fit()

In [3]:
#writes analysis/diagnostics
result.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ ns(BonusMalus, df=4) + ns(VehAge, df=4) + ns(DrivAge, df=4) + ns(VehPower, df=4) + TE(Area) + TE(Region) + TE(VehGas) + TE(VehBrand)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 42, 'df_resid': 406397, 'converged': True, 'iterations': 7}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21305.09, loss=0.309535, deviance=125807.2, log_likelihood=-83677.43, aic=167438.85, gini=0.284, auc=0.642, ae_ratio=1.0048, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1925.78, actual=0.389452, predicted=0.075353, ae_ratio=5.1683), DecileMetrics(decile=2, n=40643, exposure=4241.36, actual=0.225163, predicted=0.088822, ae_ratio=2.535), DecileMetrics(decile=3, n=40643, exposure=8288.12, actual=0.163366, predicted=0.08452, ae_ratio=1.9329), DecileMetrics(decile=4, n=40643, exposure=14053.67, actual=0.119684, predicte

In [4]:
print(result.summary())

                                 GLM Results                                  

Family:              Poisson         No. Observations:        678012
Link Function:       (default)       Df Residuals:            677970
Method:              IRLS + Smooth   Df Model:                    41
Scale:               1.0000          Alpha (λ):               0.0000
L1 Ratio:            0.00            Iterations:                   7
Non-zero coefs:      41             

Log-Likelihood:         -139176.4442 Deviance:                209352.7422
AIC:                     278436.8883 Null Deviance:           220244.0296
BIC:                     278916.8190 Pearson chi2:             1520827.82
Converged:           True           

------------------------------------------------------------------------------
Variable                  Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept              -3.1069 

In [5]:
result.predict(holdout_data)

array([0.05191895, 0.04892635, 0.04946797, ..., 0.00560013, 0.01745464,
       0.07130764], shape=(135957,))

In [6]:
# Model spec before interaction
terms = {
    "BonusMalus": {"type": "linear"},
    "VehAge": {"type": "ns", "df": 5},
    "DrivAge": {"type": "linear"},
    "Region": {"type": "target_encoding"},
    "VehBrand": {"type": "target_encoding"},
    "Area": {"type": "target_encoding"},
}
# Interaction to add (DrivAge x BonusMalus)
interactions = [
    {
        "DrivAge": {"type": "linear"},
        "Region": {"type": "target_encoding"},
        "include_main": False,
    }
]
# Fit with interaction
result = rs.glm_dict(
    response="ClaimCount",
    terms=terms,
    interactions=interactions,
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit()


In [7]:
print(result.summary())

                                 GLM Results                                  

Family:              Poisson         No. Observations:        406439
Link Function:       (default)       Df Residuals:            406428
Method:              IRLS            Df Model:                    10
Scale:               1.0000          Iterations:                   6

Log-Likelihood:          -84123.8375 Deviance:                126700.0252
AIC:                     168269.6749 Null Deviance:           132328.3090
BIC:                     168389.7420 Pearson chi2:              936937.64
Converged:           True           

------------------------------------------------------------------------------
Variable                 Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept             -4.3978     0.1585  -27.741  <0.0001   [ -4.7085,  -4.0871]  ***
BonusMalus             0.0219     0.0004   54.633  

In [8]:
#creates analysis/exploration.json
exploration = rs.explore_data(
    data=train_data,
    response="ClaimCount",
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ],
    exposure="Exposure",
)

In [9]:
#writes analysis/diagnostics
result.diagnostics(
    train_data=train_data,
    test_data = test_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ BonusMalus + ns(VehAge, df=5) + DrivAge + TE(Region) + TE(VehBrand) + TE(Area)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 11, 'df_resid': 406428, 'converged': True, 'iterations': 6}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21417.57, loss=0.31169, deviance=126682.83, log_likelihood=-84115.24, aic=168252.48, gini=0.2598, auc=0.6299, ae_ratio=0.9996, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1902.44, actual=0.411577, predicted=0.078015, ae_ratio=5.2756), DecileMetrics(decile=2, n=40643, exposure=4171.58, actual=0.238279, predicted=0.091497, ae_ratio=2.6042), DecileMetrics(decile=3, n=40643, exposure=8102.97, actual=0.164754, predicted=0.087569, ae_ratio=1.8814), DecileMetrics(decile=4, n=40643, exposure=13727.22, actual=0.127047, predicted=0.083077, ae_ratio=1.5293), DecileMetrics(decile

In [10]:
predictions = result.predict(holdout_data)

In [11]:
model_bytes = result.to_bytes()
with open("model.bin", "wb") as f:
    f.write(model_bytes)

In [12]:
with open("model.bin", "rb") as f:
    loaded = rs.GLMModel.from_bytes(f.read())

In [13]:
predictions2 = loaded.predict(holdout_data)

In [14]:
import numpy as np
print(np.array_equal(predictions2, predictions))

True


In [15]:
#writes analysis/diagnostics
loaded.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

AttributeError: '_DeserializedResult' object has no attribute 'bse'

In [ ]:
result2 = rs.glm_dict(
    response="ClaimCount",
    terms={
        "Region": {"type": "target_encoding"},
        "BonusMalus": {"type": "linear"},
        "BonusMalus2": {"type": "expression", "expr": "BonusMalus ** 2"}
    },
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit(
    regularization="elastic_net"
)

In [ ]:
#writes analysis/diagnostics
result2.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

In [ ]:
result3 = rs.glm_dict(
    response="ClaimCount",
    terms={
        "BonusMalus": {"type": "ns"},
        "VehAge": {"type": "ns"},
        "DrivAge": {"type": "linear"},
        "VehBrand": {"type": "target_encoding"}
    },
    interactions= [
        {
            "VehAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        },
        {
            "DrivAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        }
    ],
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit(
    regularization="elastic_net",
    include_unregularized=False
)

In [ ]:
import rustystats as rs

# Model spec that caused the issue when elastic_net was applied
terms = {
    "BonusMalus": {'type' : "linear"},
    "VehAge": {'type' : "ns"}, 
    "DrivAge": {'type' : "ns"},
    "Area": {'type' : "target_encoding"},
}

interactions= [
        {
            "DrivAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        }
    ]

# Build the model
glm = rs.glm_dict(
    response="ClaimCount",
    terms=terms,
    interactions=interactions,
    data=train_data,  # freMTPL2freq dataset
    family="poisson",
    offset="Exposure",  # log-linked
)

# This is where the issue occurs - regularized fit + diagnostics
result = glm.fit()

# Check if diagnostics returns None for deviance
diag = result.diagnostics(train_data=train_data, test_data=holdout_data)
print(diag.to_dict())  # Check train_test.test.deviance

{'model_summary': {'formula': 'ClaimCount ~ BonusMalus + ns(VehAge, df=4) + ns(DrivAge, df=4) + TE(Area)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 22, 'df_resid': 406417, 'converged': True, 'iterations': 7, 'regularization': {'type': 'ridge', 'alpha': 0.4, 'l1_ratio': 0.0}}, 'train_test': {'train': {'dataset': 'train', 'n_obs': 406439, 'total_exposure': 214930.9, 'total_actual': 21408.0, 'total_predicted': 21530.19, 'loss': 0.310655, 'deviance': 126262.46, 'log_likelihood': -83905.05, 'aic': 167854.11, 'gini': 0.2747, 'auc': 0.6374, 'ae_ratio': 0.9943, 'ae_by_decile': [{'decile': 1, 'n': 40643, 'exposure': 1909.44, 'actual': 0.392262, 'predicted': 0.077322, 'ae_ratio': 5.0731}, {'decile': 2, 'n': 40643, 'exposure': 4144.57, 'actual': 0.234765, 'predicted': 0.091204, 'ae_ratio': 2.5741}, {'decile': 3, 'n': 40643, 'exposure': 8115.13, 'actual': 0.167711, 'predicted': 0.087742, 'ae_ratio': 1.9114}, {'decile': 4, 'n': 40643, 'exposure': 13627.33, 'actual': 0.125263